In [ ]:
import pickle
from dataclasses import fields
from pathlib import Path

import igl
import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import base, ulac

In [ ]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = ulac.FeatureTags(MV=0, LAA=1, LIPV=2, LSPV=3, RIPV=4, RSPV=5)

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [ ]:
# segmentation_paths = ulac.construct_paths(mesh, anatomical_tags)

# segmentation_file_name.parent.mkdir(parents=True, exist_ok=True)
# with segmentation_file_name.open("wb") as f:
#     pickle.dump(segmentation_paths, f)

In [ ]:
with segmentation_file_name.open("rb") as f:
    segmentation_paths = pickle.load(f)

In [ ]:
markers = ulac.get_marker_points(segmentation_paths)
parameterized_paths = ulac.parameterize_paths(mesh, segmentation_paths, markers)
uac_paths = ulac.construct_uac_boundaries(parameterized_paths)
path_boundaries = ulac.get_patch_boundaries(uac_paths)

In [ ]:
roof_boundary = path_boundaries.roof.inds
splitted_meshes = base.split_from_enclosing_boundary(mesh, roof_boundary)
roof_mesh = splitted_meshes[1]

vertices = np.array(mesh.points[roof_mesh.point_inds])
simplices = roof_mesh.connectivity
boundary_inds = np.array([np.where(roof_mesh.point_inds == ind)[0][0] for ind in roof_boundary])
uac_coordinates = np.hstack(
    (path_boundaries.roof.alpha[:, None], path_boundaries.roof.beta[:, None])
)

In [ ]:
harmonic_map = igl.bijective_composite_harmonic_mapping(
    V=vertices,
    F=simplices,
    b=boundary_inds,
    bc=uac_coordinates,
    min_steps=10,
    max_steps=100,
    num_inner_iters=100,
    test_for_flips=True,
)
harmonic_map_z = np.zeros((harmonic_map.shape[0], 1))
harmonic_map_3d = np.hstack((harmonic_map[:, :2], harmonic_map_z))
uac_mesh = pv.PolyData.from_regular_faces(harmonic_map_3d, simplices)

In [ ]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(uac_mesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()